# Constants

In [ ]:
import os
from dotenv import load_dotenv # requires python-dotenv
# import logging
# logging.basicConfig(level=logging.INFO) # Configure logging 

load_dotenv("./../config/credentials_my.env")
model_name =  "gpt-4o-mini" # gpt-4o-mini or gpt-4o

print(f'Project Connection String: <...{os.environ["PROJECT_CONNECTION_STRING"][-30:]}>')

# Create AI Foundry Project Client

In [ ]:
from azure.ai.projects import AIProjectClient
from azure.identity import DefaultAzureCredential

project_client = AIProjectClient.from_connection_string(
    credential=DefaultAzureCredential(), conn_str=os.environ["PROJECT_CONNECTION_STRING"]
)

project_client.scope

# Extract Azure OpenAI client from the AI Foundry Project

In [ ]:
from openai import AzureOpenAI # <<<<< NOT NEEDED FOR AZURE AI FOUNDRY SDK

# using explicit type hinting for IntelliSense
azure_openai_client: AzureOpenAI = project_client.inference.get_azure_openai_client(
    api_version = os.environ.get("AZURE_OPENAI_API_VERSION"),
)

# if NOT MI --> azure_openai_client.api_key = os.environ.get("AZURE_OPENAI_API_KEY") 

print(azure_openai_client.base_url)

# Create OpenAI Assistant Agent with CodeInterpreterTool as one of the tools

In [ ]:
# First, create the code interpreter definitions...
code_interpreter_tool = {"type": "code_interpreter"}
code_interpreter_resources={"code_interpreter":{"file_ids":[]}}

# ...then, create the OpenAI Assistant Agent, attaching code interpreter
assistant_agent = azure_openai_client.beta.assistants.create(
  model=model_name,
  instructions="You are a helpful agent",
  tools=[code_interpreter_tool],
  tool_resources=code_interpreter_resources,
  temperature=1,
  top_p=1
)

# Create the thread and attach a new message to it

In [ ]:
# Create a thread
thread = azure_openai_client.beta.threads.create()

print(f"Created thread: {thread}\n")

# Add a user message to the thread
message = azure_openai_client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="""
    Could you please create a bar chart for the operating profit using 
    the following data and provide the file to me? 
    Company A: $1.2 million, Company B: $2.5 million, Company C: $3.0 million, 
    Company D: $1.8 million
    """,
)

azure_openai_client.beta.threads.messages.retrieve \
    (thread_id=thread.id, message_id=message.id)

# Run the agent syncrhonously

In [ ]:
%%time

# Run the agent
run = azure_openai_client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant_agent.id
)
print(f"Run status: {run.status}")

# Looping until the run completes or fails
import time
while run.status in ['queued', 'in_progress', 'cancelling']:
    time.sleep(1)
    run = azure_openai_client.beta.threads.runs.retrieve\
        (thread_id=thread.id, run_id=run.id)
    print(f"Run status: {run.status}")  

# Retrieve messages from the thread after the agent run execution

In [ ]:
import openai

if run.status == 'completed':
    messages = azure_openai_client.beta.threads.messages.list(thread_id=thread.id)
    messages_nr = len(messages.data)
    print(f"Here are the {messages_nr} messages:\n")
    
    for i, message in enumerate(reversed(messages.data), 1):
        j =0
        print(f"\n===== MESSAGE {i} =====")
        for c in message.content:
            j += 1
            if (type(c) is openai.types.beta.threads.image_file_content_block.ImageFileContentBlock):
                print(f"\nCONTENT {j} (ImageFileContentBlock)--> image_file id: {c.image_file.file_id}")
            elif (type(c) is openai.types.beta.threads.text_content_block.TextContentBlock):
                print(f"\nCONTENT {j} (TextContentBlock) --> Text: {c.text.value}")
                for a in c.text.annotations:
                    print(f">>> Annotation in MessageTextContent {j} of message {i}: {a.text}\n")
else:
    print(f"Sorry, I can't proceed because the run status is {run.status}")

# Delete all assistants

In [ ]:
assistants = azure_openai_client.beta.assistants.list().data
print (f"Nr. of existing agents that will be deleted: {len(assistants)}")

i = 0
for a in assistants:
    i += 1
    azure_openai_client.beta.assistants.delete(a.id)
    print(f"\n{i} - Deleted agent {a}")
    

# HIC SUNT LEONES